<a href="https://colab.research.google.com/github/kkkkkkkm/Torch/blob/main/RNN/Torch_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import torch.nn as nn

In [2]:
import torch.optim as optim
import numpy as np

In [3]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
torch.manual_seed(777)

if device == 'cuda':
  torch.cuda.manual_seed_all(777)

print('기기 사용 {}'.format(device))

기기 사용 cuda


In [4]:
train_data = dsets.MNIST(
    root = 'data',
    train = True,                         
    transform = transforms.ToTensor(), 
    download = True,            
)
test_data = dsets.MNIST(
    root = 'data', 
    train = False, 
    transform = transforms.ToTensor()
)


from torch.utils.data import DataLoader
loaders = {
    'train' : torch.utils.data.DataLoader(train_data, 
                                          batch_size=100, 
                                          shuffle=True, 
                                          num_workers=1),
    
    'test'  : torch.utils.data.DataLoader(test_data, 
                                          batch_size=100, 
                                          shuffle=True, 
                                          num_workers=1),
}
loaders

  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting data/MNIST/raw/train-images-idx3-ubyte.gz to data/MNIST/raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting data/MNIST/raw/train-labels-idx1-ubyte.gz to data/MNIST/raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting data/MNIST/raw/t10k-images-idx3-ubyte.gz to data/MNIST/raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting data/MNIST/raw/t10k-labels-idx1-ubyte.gz to data/MNIST/raw



{'test': <torch.utils.data.dataloader.DataLoader at 0x7f1546796990>,
 'train': <torch.utils.data.dataloader.DataLoader at 0x7f15473f8fd0>}

In [115]:
class RNN(nn.Module):
  
  def __init__(self, input_size, hidden_size, num_layers, num_classes):
    super(RNN, self).__init__()
    self.hidden_size = hidden_size
    self.num_layers = num_layers
    self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first = True)
    self.fc = nn.Linear(hidden_size, num_classes)


  def forward(self, x):
    h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)
    c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)

    out, hidden = self.lstm(x, (h0,c0))
    out= self.fc(out[:,:,:])

    return out




In [116]:
sequence_length = 28
input_size = 28
hidden_size = 128
num_layers = 1
num_classes = 10
batch_size = 100
num_epochs = 2
learning_rate = 0.01

model = RNN(input_size, hidden_size, num_layers, num_classes).to(device)
print(model)

RNN(
  (lstm): LSTM(28, 128, batch_first=True)
  (fc): Linear(in_features=128, out_features=10, bias=True)
)


In [117]:
loss_func = nn.CrossEntropyLoss()
loss_func

CrossEntropyLoss()

In [118]:
optimizer = optim.Adam(model.parameters(), lr =  learning_rate)
optimizer

Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    eps: 1e-08
    lr: 0.01
    weight_decay: 0
)

In [121]:
def train(num_epochs, model, loaders):
  print(f'num_epochs : {num_epochs}')
  print(f"model : {model}")
  print(f"loader : ['train'] : {loaders['train']}")

  total_step = len(loaders['train'])

  print("Started epoch : ")

  for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(loaders['train']):
      #print("epoch: ", epoch)
      #print(f"images: {images}")
      #print(f"labels: {labels}")
      images = images.reshape(-1, sequence_length, input_size).to(device)
      #print(f"images after shape: {images}")
      labels = labels.to(device)

      output = model(images)
      
      loss = loss_func(output[:,-1,:], labels) # loss = loss_func(output[:,-1,:], labels)
      
      optimizer.zero_grad()
      loss.backward()
      optimizer.step()

      if (i+1) % 100 == 0:
        print('Epoch [{}/{}], Step [{}/{}], Loss = {:4f}'.format(epoch +1, num_epochs, i+1,
                                                                 total_step, loss.item()))


  print("End epoch")
  pass 

train(num_epochs, model, loaders)

num_epochs : 2
model : RNN(
  (lstm): LSTM(28, 128, batch_first=True)
  (fc): Linear(in_features=128, out_features=10, bias=True)
)
loader : ['train'] : <torch.utils.data.dataloader.DataLoader object at 0x7f15473f8fd0>
Started epoch : 
Epoch [1/2], Step [100/600], Loss = 0.687149
Epoch [1/2], Step [200/600], Loss = 0.394108
Epoch [1/2], Step [300/600], Loss = 0.294957
Epoch [1/2], Step [400/600], Loss = 0.128097
Epoch [1/2], Step [500/600], Loss = 0.200756
Epoch [1/2], Step [600/600], Loss = 0.145592
Epoch [2/2], Step [100/600], Loss = 0.074082
Epoch [2/2], Step [200/600], Loss = 0.036881
Epoch [2/2], Step [300/600], Loss = 0.096445
Epoch [2/2], Step [400/600], Loss = 0.121009
Epoch [2/2], Step [500/600], Loss = 0.099335
Epoch [2/2], Step [600/600], Loss = 0.123717
End epoch


In [120]:
for i, (images, labels) in enumerate(loaders['train']):
      #print("epoch: ", epoch)
      print(f"images: {images.shape}")
      #print(f"labels: {labels}")
      images = images.reshape(-1, sequence_length, input_size).to(device)
      print(f"images after shape: {images.shape}")
      labels = labels.to(device)
      break

images: torch.Size([100, 1, 28, 28])
images after shape: torch.Size([100, 28, 28])


In [130]:
with torch.no_grad():
  model.eval()
  correct = 0
  total = 0
  for images, labels in loaders['test']:
    images = images.reshape(-1, sequence_length, input_size).to(device) #(image batch, -1, 28,28)  -> reshape (100,28,28)
    labels = labels.to(device)

    outputs = model(images)
    print("output shape = {}".format(outputs.shape)) # (100,10)
    what_is, predicted = torch.max(outputs[:,-1,:].data, 1) #what_is, 선택된 가장 큰 값, predicted, 인덱스 위치 .
    print("what is = {} , predicted = {}".format(what_is, predicted))
    total = total + labels.size(0) #매 batch 크기를 계속 덧셈
    print("label size = {}".format(labels.size()))
    print(f"actual_labels = {labels}")
    correct = correct + (predicted == labels).sum().item()
    break

print('Test Accuracy of the model : {} %'.format(100* correct / total))

output shape = torch.Size([100, 28, 10])
what is = tensor([ 9.6138, 11.4777,  8.8557,  7.6576, 10.4665, 10.9998,  7.4030,  8.3059,
         9.8232,  7.5440,  7.2223,  8.9326, 11.3669,  9.3379,  9.7224, 11.4419,
         9.3091,  7.1576,  7.3451,  8.5683,  7.9474,  3.0937,  8.9476,  8.6737,
         6.3164,  8.9098,  6.0258,  6.9287,  8.1011,  5.3058,  6.7561,  8.4125,
         9.4887,  6.9215, 10.3341,  9.6101,  6.0196,  5.4904,  7.2082,  8.0963,
         8.5580, 10.3973,  9.1846,  8.1599, 12.2268,  6.4816,  8.4486,  7.2238,
         6.7747,  9.1115,  8.7260,  8.4807,  8.6175,  9.5930,  6.9568, 11.6421,
         8.1300,  7.5947,  7.3474,  6.8876,  5.1780,  9.4525,  5.7249,  8.3679,
         7.4249,  9.1265,  9.0010,  8.1365, 10.6222, 11.9760,  8.3103,  6.9906,
         8.3759,  6.9765,  7.6324,  9.8774, 11.2185,  5.8165, 10.5719,  9.4345,
         9.1614,  7.0087,  7.9298,  8.8651,  5.2443,  8.2708, 10.0959,  9.1222,
         6.3006,  7.8569,  8.4116,  9.0557,  6.6148, 10.4189,  4.2221

In [124]:
sample = next(iter(loaders['test']))

In [125]:
imgs, lbls = sample

In [126]:
imgs = imgs.to(device)
lbls= lbls.to(device)

In [149]:
test_output = model(imgs[:10].view(-1,28,28))
test_predict = torch.argmax(test_output[:,25,:],1)
test_lbls = lbls[:10]

print(f"Test_predict = {test_predict}")
print(f"actual_label = {test_lbls}")


Test_predict = tensor([0, 1, 9, 2, 2, 2, 9, 4, 4, 5], device='cuda:0')
actual_label = tensor([0, 1, 9, 2, 2, 2, 8, 4, 4, 5], device='cuda:0')


In [135]:
test_output.shape

torch.Size([10, 28, 10])

In [145]:
test_predict.shape

torch.Size([10, 28])

torch.Size([])